In [1]:
import shapely.geometry
import geopandas as gpd

In [2]:
from gtfs_utils import load_gtfs

In [3]:
df_dict = load_gtfs("/Users/luis/Developer/PyCharm/upscore/data/germany-latest")

In [4]:
dic = df_dict["stops"][["stop_lon", "stop_lat", "stop_id"]].compute()
d = gpd.GeoDataFrame(
    dic["stop_id"], geometry=gpd.points_from_xy(dic.stop_lon, dic.stop_lat)
)
del dic

,stop_id,geometry
0,909684,POINT (6.24734 51.87225)
1,167505,POINT (6.24727 51.87222)
2,815255,POINT (6.24741 51.87228)
3,691093,POINT (6.24741 51.87228)
4,127587,POINT (6.24727 51.87222)
...,...,...
1146336,203886,POINT (15.84793 50.40331)
1146337,567221,POINT (15.84793 50.40331)
1146338,663978,POINT (18.69595 45.07754)
1146339,88106,POINT (18.69595 45.07754)


In [5]:
geom = shapely.geometry.box(
    *[
        8.88530041045740746562,
        46.34379183997860392404,
        14.27013758954259253438,
        49.93713016002139607596,
    ]
)

In [6]:
mask = d.within(geom)
d = d[mask]

In [7]:
# filter stops.txt -
stop_ids = d["stop_id"].values
mask = df_dict["stops"]["stop_id"].isin(stop_ids)
df_dict["stops"][mask].to_csv("test/stops.txt", single_file=True)

['/Users/luis/Developer/PyCharm/gtfsutils/test/stops.txt']

In [8]:
# filter transfers.txt
if "transfers" in df_dict:
    mask = df_dict["transfers"]["from_stop_id"].isin(stop_ids) & df_dict["transfers"][
        "to_stop_id"
    ].isin(stop_ids)
    df_dict["transfers"] = df_dict["transfers"][mask]
    df_dict["transfers"].to_csv("test/transfers.txt", single_file=True)

In [9]:
%%time

# filter stop_times.txt -
mask = df_dict["stop_times"]["stop_id"].isin(stop_ids)
unique_trip_ids = df_dict["stop_times"][mask]["trip_id"].unique()
trip_ids = unique_trip_ids.values.compute()
mask = df_dict["stop_times"]["trip_id"].isin(trip_ids)
df_dict["stop_times"][mask].to_csv("test/stop_times.txt", single_file=True)

CPU times: user 6min 56s, sys: 1min 40s, total: 8min 37s
Wall time: 4min 9s


['/Users/luis/Developer/PyCharm/gtfsutils/test/stop_times.txt']

In [10]:
%%time

# filter trips.txt -
mask = df_dict["trips"]["trip_id"].isin(trip_ids)
df_dict["trips"] = df_dict["trips"][mask]
df_dict["trips"].to_csv("test/trips.txt", single_file=True)

del trip_ids

CPU times: user 13.7 s, sys: 3.3 s, total: 17 s
Wall time: 10.3 s


['/Users/luis/Developer/PyCharm/gtfsutils/test/trips.txt']

In [11]:
%%time

# Filter route.txt
route_ids = df_dict["trips"]["route_id"].values
mask = df_dict["routes"]["route_id"].isin(route_ids.compute())
df_dict["routes"] = df_dict["routes"][mask]
df_dict["routes"].to_csv("test/routes.txt", single_file=True)

CPU times: user 8 s, sys: 2.88 s, total: 10.9 s
Wall time: 3.98 s


['/Users/luis/Developer/PyCharm/gtfsutils/test/routes.txt']

In [12]:
%%time

# Filter agency.txt
agency_ids = df_dict["routes"]["agency_id"].values
mask = df_dict["agency"]["agency_id"].isin(agency_ids.compute())
df_dict["agency"] = df_dict["agency"][mask]
df_dict["agency"].to_csv("test/agency.txt", single_file=True)

CPU times: user 40 ms, sys: 20.1 ms, total: 60.1 ms
Wall time: 58.5 ms


['/Users/luis/Developer/PyCharm/gtfsutils/test/agency.txt']